In [1]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    print("Columns after renaming:", df.columns)


    # Calculate Total Cost if missing
    # if 'Total_Cost' not in df.columns:
    #     df['Total_Cost'] = df['Insurance_Cost'] + df['Maintenance_Cost'] + df['Fuel_Costs']

    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'Cost ($)':row['Cost'],
                'fuel_costs_per_km': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            # Get demand for this combination
            demand = vehicles[0]['demand']  # All vehicles in the same group have the same demand
            
            # Calculate max vehicles based on the vehicle with highest yearly range
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            
            # Calculate max vehicles needed and round up to ensure demand is met
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        
        return max_vehicles

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the total number of vehicles in the solution is within the dynamic maximum limit"""
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    def calculate_total_cost(self, num_vehicles: int, vehicle: Dict) -> float:
        """Calculate total cost including insurance, maintenance, and fuel costs"""
        return num_vehicles * (
            vehicle['insurance_cost'] + 
            vehicle['maintenance_cost'] + 
            vehicle['fuel_costs_per_km']+
            vehicle['Cost ($)']
        )

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            # Generate a solution
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            
            # Randomly distribute vehicles while respecting the total maximum
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:  # 50% chance to add a vehicle
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions while respecting dynamic vehicle limit"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        # If we can't create valid children, return copies of parents
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.1) -> Dict:
        """Mutate a solution while respecting the dynamic vehicle limit"""
        attempts = 0
        max_attempts = 10
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            total_vehicles = sum(mutated_solution.values())
            
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if (change == 1 and total_vehicles < max_vehicles) or \
                       (change == -1 and mutated_solution[vehicle_type] > 0):
                        mutated_solution[vehicle_type] += change
                        total_vehicles += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution  # Return original if no valid mutation found

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness of a solution based on:
        1. TOPSIS score
        2. Meeting demand requirements
        3. Total cost
        4. Penalty for exceeding maximum vehicles
        """
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_cost = 0
        total_capacity = 0
        weighted_topsis = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_cost += self.calculate_total_cost(num_vehicles, vehicle)
            total_capacity += num_vehicles * vehicle['yearly_range']
            weighted_topsis += num_vehicles * vehicle['topsis_score']
        
        # Penalize solutions that don't meet demand
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        # Normalize costs (lower is better)
        cost_score = 1 / (total_cost + 1)
        
        return weighted_topsis + cost_score - demand_penalty

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            # Calculate fitness for all solutions
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            # Sort by fitness (descending)
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            # Update best solution if needed
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            # Select parents for next generation
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            # Create next generation
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results in the required format"""
        results = []
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    total_cost = self.calculate_total_cost(num_vehicles, vehicle_data)

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Cost ($)": round(total_cost, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        # "Total_Cost":vehicle_data['Total_Cost']
                    })

        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with dynamic maximum vehicles...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"  # Replace with your CSV file path
    results_df = main(csv_path)
    results_df
    results_df.to_csv("optimized_fleet_allocation_2023.csv", index=False)


Running optimization with dynamic maximum vehicles...
Columns after renaming: Index(['Allocation', 'Operating Year', 'Size', 'Distance_demand', 'Demand',
       'ID', 'Vehicle', 'Available Year', 'Cost', 'Yearly_Range',
       'Distance_vehicle', 'Fuel', 'carbon_emissions_per_km', 'Insurance_Cost',
       'Maintenance_Cost', 'Fuel_Costs', 'Total_Cost', 'Topsis_Score', 'Rank',
       'Distance'],
      dtype='object')

Optimized Fleet Allocation:
              Allocation Vehicle    Cost ($)         Fuel  no_of_vehicles  \
0   Size S1, Distance D1     LNG   954001.51          LNG               9   
1   Size S1, Distance D2  Diesel   720802.18          B20               8   
2   Size S1, Distance D2     LNG  1908003.01          LNG              18   
3   Size S1, Distance D3  Diesel  1081203.27          B20              12   
4   Size S1, Distance D3     LNG  2226003.52          LNG              21   
5   Size S1, Distance D4     LNG   530000.84          LNG               5   
6   Size S

In [ ]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

def load_previous_optimization(csv_path: str) -> pd.DataFrame:
    """Load the previously optimized fleet allocation results."""
    try:
        return pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"Warning: Previous optimization file {csv_path} not found.")
        return None

def extract_size_number(size_str: str) -> int:
    """Extract the numeric part from size string (e.g., 'S1' -> 1)"""
    return int(size_str.replace('S', ''))

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame, previous_optimization: pd.DataFrame = None):
        self.data = data
        self.previous_optimization = previous_optimization
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.previous_allocation = self._process_previous_allocation()
    
    def _process_previous_allocation(self) -> Dict:
        """Process previous year's allocation into a usable format"""
        if self.previous_optimization is None:
            return {}
        
        previous_allocation = {}
        for _, row in self.previous_optimization.iterrows():
            try:
                # Extract size and distance from allocation string
                allocation_parts = row['Allocation'].split(', ')
                # Handle 'Size S1' format
                size_str = allocation_parts[0].split(' ')[1]  # Gets 'S1'
                size = extract_size_number(size_str)  # Converts 'S1' to 1
                distance = allocation_parts[1].split(' ')[1]
                
                key = (size, distance)
                if key not in previous_allocation:
                    previous_allocation[key] = {}
                
                previous_allocation[key][row['Vehicle']] = {
                    'num_vehicles': row['no_of_vehicles'],
                    'yearly_range': row['Yearly Range'],
                    'demand': row['Demand']
                }
            except (IndexError, ValueError) as e:
                print(f"Warning: Could not process row {row['Allocation']}: {e}")
                continue
        
        return previous_allocation

    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'Cost ($)': row['Cost'],
                'fuel_costs_per_km': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population with bias towards previous year's allocation"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        # Add previous year's allocation if available
        if size_distance in self.previous_allocation:
            prev_solution = {v['vehicle_type']: 0 for v in vehicles}
            for vehicle_type, data in self.previous_allocation[size_distance].items():
                if vehicle_type in prev_solution:
                    prev_solution[vehicle_type] = data['num_vehicles']
            if self.is_valid_solution(prev_solution, size_distance):
                population.append(prev_solution)
        
        # Generate rest of the population
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    def calculate_total_cost(self, num_vehicles: int, vehicle: Dict) -> float:
        return num_vehicles * (
            vehicle['insurance_cost'] + 
            vehicle['maintenance_cost'] + 
            vehicle['fuel_costs_per_km'] +
            vehicle['Cost ($)']
        )

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """Enhanced fitness function considering previous year's allocation"""
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_cost = 0
        total_capacity = 0
        weighted_topsis = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_cost += self.calculate_total_cost(num_vehicles, vehicle)
            total_capacity += num_vehicles * vehicle['yearly_range']
            weighted_topsis += num_vehicles * vehicle['topsis_score']
        
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        cost_score = 1 / (total_cost + 1)
        
        alignment_bonus = 0
        if size_distance in self.previous_allocation:
            prev_allocation = self.previous_allocation[size_distance]
            for vehicle_type, num_vehicles in solution.items():
                if vehicle_type in prev_allocation:
                    prev_num = prev_allocation[vehicle_type]['num_vehicles']
                    alignment_bonus += 10 / (abs(num_vehicles - prev_num) + 1)
        
        return weighted_topsis + cost_score - demand_penalty + alignment_bonus

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.1) -> Dict:
        """Mutate a solution"""
        attempts = 0
        max_attempts = 10
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            total_vehicles = sum(mutated_solution.values())
            
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if (change == 1 and total_vehicles < max_vehicles) or \
                       (change == -1 and mutated_solution[vehicle_type] > 0):
                        mutated_solution[vehicle_type] += change
                        total_vehicles += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results with year-over-year comparison"""
        results = []
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    total_cost = self.calculate_total_cost(num_vehicles, vehicle_data)
                    
                    # Get previous year's allocation for comparison
                    prev_vehicles = 0
                    if (size_distance in self.previous_allocation and 
                        vehicle_type in self.previous_allocation[size_distance]):
                        prev_vehicles = self.previous_allocation[size_distance][vehicle_type]['num_vehicles']

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Cost ($)": round(total_cost, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Previous_Year_Vehicles": prev_vehicles,
                        "Vehicle_Change": num_vehicles - prev_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range']
                    })

        return pd.DataFrame(results)

def main(csv_path_2024: str, previous_optimization_path: str):
    """Main function to run the optimization with year-over-year comparison"""
    print("\nLoading 2024 data and previous optimization results...")
    data_df_2024 = load_and_preprocess_data(csv_path_2024)
    previous_optimization = load_previous_optimization(previous_optimization_path)
    
    print("\nRunning optimization for 2024 fleet...")
    optimizer = FleetOptimizer(data_df_2024, previous_optimization)
    optimized_results = optimizer.get_optimized_results()
    
    print("\nOptimized Fleet Allocation for 2024:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path_2024 = "topsis_result/topsis_results_2024.csv"
    previous_optimization_path = "optimized_fleet_allocation.csv"
    results_df = main(csv_path_2024, previous_optimization_path)
    results_df.to_csv("optimized_fleet_allocation_2024.csv", index=False)


Loading 2024 data and previous optimization results...

Running optimization for 2024 fleet...

Optimized Fleet Allocation for 2024:
              Allocation Vehicle    Cost ($)         Fuel  no_of_vehicles  \
0   Size S1, Distance D1     LNG   982621.50          LNG               9   
1   Size S1, Distance D2  Diesel  1113639.34          B20              12   
2   Size S1, Distance D2     LNG  1637702.50          LNG              15   
3   Size S1, Distance D3  Diesel  1299245.89          B20              14   
4   Size S1, Distance D3     LNG  2074423.16          LNG              19   
5   Size S1, Distance D4     LNG   545900.83          LNG               5   
6   Size S2, Distance D1  Diesel   340642.44          B20               3   
7   Size S2, Distance D1     LNG  1016466.96          LNG               7   
8   Size S2, Distance D2  Diesel   454189.92          B20               4   
9   Size S2, Distance D2     LNG  1452095.66          LNG              10   
10  Size S2, Distan

In [2]:
import pandas as pd
import numpy as np
from typing import Tuple, Dict

def compare_fleet_allocations(results_2023: pd.DataFrame, results_2024: pd.DataFrame) -> pd.DataFrame:
    """
    Compare fleet allocations between 2023 and 2024 and identify improvements
    
    Parameters:
    results_2023: DataFrame with 2023 optimization results
    results_2024: DataFrame with 2024 optimization results
    
    Returns:
    DataFrame with side-by-side comparison and improvement metrics
    """
    # Create a unique identifier for each size-distance combination
    def extract_size_distance(allocation: str) -> Tuple[str, str]:
        size = allocation.split(',')[0].replace('Size ', '').strip()
        distance = allocation.split(',')[1].replace('Distance ', '').strip()
        return (size, distance)
    
    # Process each year's results
    def process_results(df: pd.DataFrame) -> Dict:
        processed = {}
        for _, row in df.iterrows():
            key = extract_size_distance(row['Allocation'])
            if key not in processed:
                processed[key] = {
                    'total_cost': 0,
                    'total_vehicles': 0,
                    'vehicles': [],
                    'demand': row['Demand'],
                    'yearly_range': 0
                }
            processed[key]['total_cost'] += row['Cost ($)']
            processed[key]['total_vehicles'] += row['no_of_vehicles']
            processed[key]['vehicles'].append(f"{row['Vehicle']} ({row['no_of_vehicles']})")
            processed[key]['yearly_range'] += row['Yearly Range'] * row['no_of_vehicles']
        return processed

    results_2023_proc = process_results(results_2023)
    results_2024_proc = process_results(results_2024)

    # Create comparison DataFrame
    comparison_data = []
    all_keys = set(results_2023_proc.keys()) | set(results_2024_proc.keys())
    
    for key in all_keys:
        data_2023 = results_2023_proc.get(key, {})
        data_2024 = results_2024_proc.get(key, {})
        
        if data_2023 and data_2024:
            cost_diff = data_2024['total_cost'] - data_2023['total_cost']
            cost_pct_change = (cost_diff / data_2023['total_cost']) * 100
            vehicle_diff = data_2024['total_vehicles'] - data_2023['total_vehicles']
            
            comparison_data.append({
                'Size': key[0],
                'Distance': key[1],
                '2023_Cost': data_2023['total_cost'],
                '2024_Cost': data_2024['total_cost'],
                'Cost_Difference': cost_diff,
                'Cost_Change_%': cost_pct_change,
                '2023_Vehicles': data_2023['total_vehicles'],
                '2024_Vehicles': data_2024['total_vehicles'],
                'Vehicle_Difference': vehicle_diff,
                '2023_Fleet_Mix': ', '.join(data_2023['vehicles']),
                '2024_Fleet_Mix': ', '.join(data_2024['vehicles']),
                'Demand': data_2023['demand'],
                '2023_Yearly_Range': data_2023['yearly_range'],
                '2024_Yearly_Range': data_2024['yearly_range']
            })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df.sort_values(['Size', 'Distance'], inplace=True)
    
    # Add analysis flags
    comparison_df['Better_In_2024'] = comparison_df.apply(
        lambda x: 'Yes' if x['Cost_Difference'] < 0 and 
                          x['2024_Yearly_Range'] >= x['Demand'] else 'No', 
        axis=1
    )
    
    return comparison_df

def main():
    # Load the optimization results
    results_2023 = pd.read_csv("optimized_fleet_allocation_2023.csv")
    results_2024 = pd.read_csv("optimized_fleet_allocation_2024.csv")
    
    # Compare the results
    comparison = compare_fleet_allocations(results_2023, results_2024)
    
    # Display summary of improvements
    print("\nFleet Allocation Comparison Summary:")
    print(f"Total combinations analyzed: {len(comparison)}")
    improved_count = len(comparison[comparison['Better_In_2024'] == 'Yes'])
    print(f"Combinations with improvements in 2024: {improved_count}")
    
    # Display detailed improvements
    print("\nDetailed improvements (where 2024 allocation is better):")
    improvements = comparison[comparison['Better_In_2024'] == 'Yes'].copy()
    improvements['Cost_Savings'] = -improvements['Cost_Difference']
    
    if len(improvements) > 0:
        print(improvements[[
            'Size', 'Distance', '2023_Cost', '2024_Cost', 
            'Cost_Savings', '2023_Fleet_Mix', '2024_Fleet_Mix'
        ]].to_string())
    else:
        print("No combinations showed improvement in 2024")
    
    # Save detailed comparison
    comparison.to_csv("fleet_allocation_comparison.csv", index=False)
    return comparison

if __name__ == "__main__":
    comparison_results = main()


Fleet Allocation Comparison Summary:
Total combinations analyzed: 16
Combinations with improvements in 2024: 1

Detailed improvements (where 2024 allocation is better):
  Size Distance  2023_Cost   2024_Cost  Cost_Savings                  2023_Fleet_Mix                  2024_Fleet_Mix
3   S3       D1  5293964.1  4976055.66     317908.44  BEV (5), Diesel (10), LNG (15)  BEV (2), Diesel (12), LNG (17)


In [1]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    print("Columns after renaming:", df.columns)


    # Calculate Total Cost if missing
    # if 'Total_Cost' not in df.columns:
    #     df['Total_Cost'] = df['Insurance_Cost'] + df['Maintenance_Cost'] + df['Fuel_Costs']

    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'Cost ($)':row['Cost'],
                'fuel_costs_per_km': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            # Get demand for this combination
            demand = vehicles[0]['demand']  # All vehicles in the same group have the same demand
            
            # Calculate max vehicles based on the vehicle with highest yearly range
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            
            # Calculate max vehicles needed and round up to ensure demand is met
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        
        return max_vehicles

    def calculate_utilization(self, demand: float, num_vehicles: int, yearly_range: float) -> float:
        """Calculate utilization metric for a vehicle type"""
        if num_vehicles == 0 or yearly_range == 0:
            return 0
        return (demand / num_vehicles) / yearly_range * 100

    def calculate_demand_fulfillment(self, num_vehicles: int, max_vehicles: int) -> float:
        """Calculate demand fulfillment by fuel type"""
        if max_vehicles == 0:
            return 0
        return num_vehicles / max_vehicles

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the total number of vehicles in the solution is within the dynamic maximum limit"""
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    def calculate_total_cost(self, num_vehicles: int, vehicle: Dict) -> float:
        """Calculate total cost including insurance, maintenance, and fuel costs"""
        return num_vehicles * (
            vehicle['insurance_cost'] + 
            vehicle['maintenance_cost'] + 
            vehicle['fuel_costs_per_km']+
            vehicle['Cost ($)']
        )

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            # Generate a solution
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            
            # Randomly distribute vehicles while respecting the total maximum
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:  # 50% chance to add a vehicle
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions while respecting dynamic vehicle limit"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        # If we can't create valid children, return copies of parents
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.1) -> Dict:
        """Mutate a solution while respecting the dynamic vehicle limit"""
        attempts = 0
        max_attempts = 10
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            total_vehicles = sum(mutated_solution.values())
            
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if (change == 1 and total_vehicles < max_vehicles) or \
                       (change == -1 and mutated_solution[vehicle_type] > 0):
                        mutated_solution[vehicle_type] += change
                        total_vehicles += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution  # Return original if no valid mutation found

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness of a solution based on:
        1. TOPSIS score
        2. Meeting demand requirements
        3. Total cost
        4. Penalty for exceeding maximum vehicles
        """
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_cost = 0
        total_capacity = 0
        weighted_topsis = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_cost += self.calculate_total_cost(num_vehicles, vehicle)
            total_capacity += num_vehicles * vehicle['yearly_range']
            weighted_topsis += num_vehicles * vehicle['topsis_score']
        
        # Penalize solutions that don't meet demand
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        # Normalize costs (lower is better)
        cost_score = 1 / (total_cost + 1)
        
        return weighted_topsis + cost_score - demand_penalty

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            # Calculate fitness for all solutions
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            # Sort by fitness (descending)
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            # Update best solution if needed
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            # Select parents for next generation
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            # Create next generation
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results in the required format"""
        results = []
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    total_cost = self.calculate_total_cost(num_vehicles, vehicle_data)

                    # Calculate evaluation metrics
                    utilization = self.calculate_utilization(
                        vehicle_data['demand'],
                        num_vehicles,
                        vehicle_data['yearly_range']
                    )
                    
                    demand_fulfillment = self.calculate_demand_fulfillment(
                        num_vehicles,
                        max_vehicles
                    )

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Cost ($)": round(total_cost, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Utilization": round(utilization, 2),
                        "Demand_Fulfillment": round(demand_fulfillment, 2)
                    })

        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with dynamic maximum vehicles...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"  # Replace with your CSV file path
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_2023.csv", index=False)


Running optimization with dynamic maximum vehicles...
Columns after renaming: Index(['Allocation', 'Operating Year', 'Size', 'Distance_demand', 'Demand',
       'ID', 'Vehicle', 'Available Year', 'Cost', 'Yearly_Range',
       'Distance_vehicle', 'Fuel', 'carbon_emissions_per_km', 'Insurance_Cost',
       'Maintenance_Cost', 'Fuel_Costs', 'Total_Cost', 'Topsis_Score', 'Rank',
       'Distance'],
      dtype='object')

Optimized Fleet Allocation:
              Allocation Vehicle    Cost ($)         Fuel  no_of_vehicles  \
0   Size S1, Distance D1     LNG   954001.51          LNG               9   
1   Size S1, Distance D2  Diesel   901002.72          B20              10   
2   Size S1, Distance D2     LNG  1696002.68          LNG              16   
3   Size S1, Distance D3  Diesel  1351504.08          B20              15   
4   Size S1, Distance D3     LNG  1908003.01          LNG              18   
5   Size S1, Distance D4     LNG   530000.84          LNG               5   
6   Size S

In [ ]:
# DONE DONE DONE!!!!!!!1

In [ ]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math
import os


def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    # print("Columns after renaming:", df.columns)


    # Calculate Total Cost if missing
    # if 'Total_Cost' not in df.columns:
    #     df['Total_Cost'] = df['Insurance_Cost'] + df['Maintenance_Cost'] + df['Fuel_Costs']

    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'Cost ($)':row['Cost'],
                'fuel_costs_per_km': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            # Get demand for this combination
            demand = vehicles[0]['demand']  # All vehicles in the same group have the same demand
            
            # Calculate max vehicles based on the vehicle with highest yearly range
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            
            # Calculate max vehicles needed and round up to ensure demand is met
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        
        return max_vehicles

    def calculate_utilization(self, demand: float, num_vehicles: int, yearly_range: float) -> float:
        """Calculate utilization metric for a vehicle type"""
        if num_vehicles == 0 or yearly_range == 0:
            return 0
        return (demand / num_vehicles) / yearly_range * 100

    def calculate_demand_fulfillment(self, num_vehicles: int, max_vehicles: int) -> float:
        """Calculate demand fulfillment by fuel type"""
        if max_vehicles == 0:
            return 0
        return num_vehicles / max_vehicles

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the total number of vehicles in the solution is within the dynamic maximum limit"""
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    # def calculate_total_cost(self, num_vehicles: int, vehicle: Dict) -> float:
    #     """Calculate total cost including insurance, maintenance, and fuel costs"""
    #     return num_vehicles * (
    #         vehicle['insurance_cost'] + 
    #         vehicle['maintenance_cost'] + 
    #         vehicle['fuel_costs_per_km']+
    #         vehicle['Cost ($)']
    #     )

    def calculate_total_cost(self, num_vehicles: int, vehicle: Dict) -> float:
        """Calculate total cost including insurance, maintenance, and fuel costs"""
        if num_vehicles == 0:
            return 0
        return num_vehicles * (
            vehicle['insurance_cost'] + 
            vehicle['maintenance_cost'] + 
            vehicle['Cost ($)']
        ) + vehicle['fuel_costs_per_km'] * (vehicle['demand'] / num_vehicles)


    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            # Generate a solution
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            
            # Randomly distribute vehicles while respecting the total maximum
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:  # 50% chance to add a vehicle
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions while respecting dynamic vehicle limit"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        # If we can't create valid children, return copies of parents
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.1) -> Dict:
        """Mutate a solution while respecting the dynamic vehicle limit"""
        attempts = 0
        max_attempts = 10
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            total_vehicles = sum(mutated_solution.values())
            
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if (change == 1 and total_vehicles < max_vehicles) or \
                       (change == -1 and mutated_solution[vehicle_type] > 0):
                        mutated_solution[vehicle_type] += change
                        total_vehicles += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution  # Return original if no valid mutation found

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness of a solution based on:
        1. TOPSIS score
        2. Meeting demand requirements
        3. Total cost
        4. Penalty for exceeding maximum vehicles
        """
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_cost = 0
        total_capacity = 0
        weighted_topsis = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_cost += self.calculate_total_cost(num_vehicles, vehicle)
            total_capacity += num_vehicles * vehicle['yearly_range']
            weighted_topsis += num_vehicles * vehicle['topsis_score']
        
        # Penalize solutions that don't meet demand
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        # Normalize costs (lower is better)
        cost_score = 1 / (total_cost + 1)
        
        return weighted_topsis + cost_score - demand_penalty

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            # Calculate fitness for all solutions
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            # Sort by fitness (descending)
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            # Update best solution if needed
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            # Select parents for next generation
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            # Create next generation
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results in the required format"""
        results = []
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    total_cost = self.calculate_total_cost(num_vehicles, vehicle_data)

                    # Calculate evaluation metrics
                    utilization = self.calculate_utilization(
                        vehicle_data['demand'],
                        num_vehicles,
                        vehicle_data['yearly_range']
                    )
                    
                    demand_fulfillment = self.calculate_demand_fulfillment(
                        num_vehicles,
                        max_vehicles
                    )

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Cost ($)": round(total_cost, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Utilization": round(utilization, 2),
                        "Demand_Fulfillment": round(demand_fulfillment, 2)
                    })

        return pd.DataFrame(results)


def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with dynamic maximum vehicles...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    
    # Calculate and print the total cost across all allocations
    total_cost = optimized_results["Cost ($)"].sum()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    print(f"\nTotal Fleet Cost: ${total_cost:,.2f}")
    
    return optimized_results

def process_all_years(input_folder: str, output_folder: str):
    """Process all CSV files in the input folder and save results in the output folder."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)  # Create output directory if it doesn't exist
    
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]  # List all CSV files
    yearly_costs = []  # Store total cost for each year
    
    for csv_file in csv_files:
        year = csv_file.split('_')[-1].replace('.csv', '')  # Extract year from filename
        csv_path = os.path.join(input_folder, csv_file)
        
        print(f"\nProcessing {csv_file}...")
        results_df = main(csv_path)  # Run optimization
        
        # Calculate total fleet cost
        total_cost = results_df["Cost ($)"].sum()
        yearly_costs.append({"Year": year, "Total Cost ($)": round(total_cost, 2)})
        
        # Save results
        output_file = os.path.join(output_folder, f"optimized_fleet_allocation_{year}.csv")
        results_df.to_csv(output_file, index=False)
        print(f"Results saved to {output_file}")
    
    # Save yearly total cost summary
    # summary_df = pd.DataFrame(yearly_costs)
    # summary_file = os.path.join(output_folder, "yearly_total_cost_summary.csv")
    # summary_df.to_csv(summary_file, index=False)
    # print(f"\nYearly total cost summary saved to {summary_file}")


# def process_all_years(input_folder: str, output_folder: str):
#     """Process all CSV files in the input folder and save results in the output folder."""
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)  # Create output directory if it doesn't exist
    
#     csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]  # List all CSV files
    
#     for csv_file in csv_files:
#         year = csv_file.split('_')[-1].replace('.csv', '')  # Extract year from filename
#         csv_path = os.path.join(input_folder, csv_file)
        
#         print(f"\nProcessing {csv_file}...")
#         results_df = main(csv_path)  # Run optimization
        
#         # Save results
#         output_file = os.path.join(output_folder, f"optimized_fleet_allocation_{year}.csv")
#         results_df.to_csv(output_file, index=False)
#         print(f"Results saved to {output_file}")


if __name__ == "__main__":
    input_folder = "topsis_result"
    output_folder = "fleet_optimization_results"
    process_all_years(input_folder, output_folder)


# if __name__ == "__main__":
#     csv_path = "topsis_result/topsis_results_2023.csv"  # Replace with your CSV file path
#     results_df = main(csv_path)
#     # results_df.to_csv("optimized_fleet_allocation_2023.csv", index=False)


Processing topsis_results_2023.csv...

Running optimization with dynamic maximum vehicles...

Optimized Fleet Allocation:
              Allocation Vehicle    Cost ($) Fuel  no_of_vehicles  \
0   Size S1, Distance D1  Diesel   298524.59  B20               2   
1   Size S1, Distance D1     LNG   762789.59  LNG               7   
2   Size S1, Distance D2  Diesel   971710.26  B20              10   
3   Size S1, Distance D2     LNG  1723177.00  LNG              16   
4   Size S1, Distance D3  Diesel  1072582.32  B20              11   
5   Size S1, Distance D3     LNG  2357053.73  LNG              22   
6   Size S1, Distance D4     LNG   543873.77  LNG               5   
7   Size S2, Distance D1     LNG  1426443.93  LNG              10   
8   Size S2, Distance D2     LNG  1990235.27  LNG              14   
9   Size S2, Distance D3  Diesel   322859.84  B20               1   
10  Size S2, Distance D3     LNG  1005438.73  LNG               7   
11  Size S2, Distance D4     LNG   293132.66  LNG

In [ ]:
# without topsis


import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    # print("Columns after renaming:", df.columns)


    # Calculate Total Cost if missing
    # if 'Total_Cost' not in df.columns:
    #     df['Total_Cost'] = df['Insurance_Cost'] + df['Maintenance_Cost'] + df['Fuel_Costs']

    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'Cost ($)':row['Cost'],
                'fuel_costs_per_km': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            # Get demand for this combination
            demand = vehicles[0]['demand']  # All vehicles in the same group have the same demand
            
            # Calculate max vehicles based on the vehicle with highest yearly range
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            
            # Calculate max vehicles needed and round up to ensure demand is met
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        
        return max_vehicles

    def calculate_utilization(self, demand: float, num_vehicles: int, yearly_range: float) -> float:
        """Calculate utilization metric for a vehicle type"""
        if num_vehicles == 0 or yearly_range == 0:
            return 0
        return (demand / num_vehicles) / yearly_range * 100

    def calculate_demand_fulfillment(self, num_vehicles: int, max_vehicles: int) -> float:
        """Calculate demand fulfillment by fuel type"""
        if max_vehicles == 0:
            return 0
        return num_vehicles / max_vehicles

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the total number of vehicles in the solution is within the dynamic maximum limit"""
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    def calculate_total_cost(self, num_vehicles: int, vehicle: Dict) -> float:
        """Calculate total cost including insurance, maintenance, and fuel costs"""
        return num_vehicles * (
            vehicle['insurance_cost'] + 
            vehicle['maintenance_cost'] + 
            vehicle['fuel_costs_per_km']+
            vehicle['Cost ($)']
        )

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            # Generate a solution
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            
            # Randomly distribute vehicles while respecting the total maximum
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:  # 50% chance to add a vehicle
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions while respecting dynamic vehicle limit"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        # If we can't create valid children, return copies of parents
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.1) -> Dict:
        """Mutate a solution while respecting the dynamic vehicle limit"""
        attempts = 0
        max_attempts = 10
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            total_vehicles = sum(mutated_solution.values())
            
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if (change == 1 and total_vehicles < max_vehicles) or \
                       (change == -1 and mutated_solution[vehicle_type] > 0):
                        mutated_solution[vehicle_type] += change
                        total_vehicles += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution  # Return original if no valid mutation found

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness of a solution based on:
        1. Meeting demand requirements
        2. Total cost
        3. Penalty for exceeding maximum vehicles
        
        TOPSIS score is now removed from the calculation
        """
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_cost = 0
        total_capacity = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_cost += self.calculate_total_cost(num_vehicles, vehicle)
            total_capacity += num_vehicles * vehicle['yearly_range']

        
        # Penalize solutions that don't meet demand
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        # Normalize costs (lower is better)
        cost_score = 1 / (total_cost + 1)
        
        # Return fitness score without TOPSIS component
        return cost_score - demand_penalty

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            # Calculate fitness for all solutions
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            # Sort by fitness (descending)
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            # Update best solution if needed
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            # Select parents for next generation
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            # Create next generation
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results in the required format"""
        results = []
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    total_cost = self.calculate_total_cost(num_vehicles, vehicle_data)

                    # Calculate evaluation metrics
                    utilization = self.calculate_utilization(
                        vehicle_data['demand'],
                        num_vehicles,
                        vehicle_data['yearly_range']
                    )
                    
                    demand_fulfillment = self.calculate_demand_fulfillment(
                        num_vehicles,
                        max_vehicles
                    )

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Cost ($)": round(total_cost, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Utilization": round(utilization, 2),
                        "Demand_Fulfillment": round(demand_fulfillment, 2)
                    })

        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with dynamic maximum vehicles...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    
    # Calculate and print the total cost across all allocations
    total_cost = optimized_results["Cost ($)"].sum()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    print(f"\nTotal Fleet Cost: ${total_cost:,.2f}")
    
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"  # Replace with your CSV file path
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_2023.csv", index=False)


Running optimization with dynamic maximum vehicles...
Columns after renaming: Index(['Allocation', 'Operating Year', 'Size', 'Distance_demand', 'Demand',
       'ID', 'Vehicle', 'Available Year', 'Cost', 'Yearly_Range',
       'Distance_vehicle', 'Fuel', 'carbon_emissions_per_km', 'Insurance_Cost',
       'Maintenance_Cost', 'Fuel_Costs', 'Total_Cost', 'Topsis_Score', 'Rank',
       'Distance'],
      dtype='object')

Optimized Fleet Allocation:
              Allocation Vehicle    Cost ($) Fuel  no_of_vehicles  \
0   Size S1, Distance D1  Diesel   450501.36  B20               5   
1   Size S1, Distance D2  Diesel   540601.63  B20               6   
2   Size S1, Distance D2     LNG   742001.17  LNG               7   
3   Size S1, Distance D3  Diesel  1081203.27  B20              12   
4   Size S1, Distance D3     LNG   530000.84  LNG               5   
5   Size S1, Distance D4  Diesel   270300.82  B20               3   
6   Size S2, Distance D1  Diesel   551201.37  B20               5 

In [4]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    print("Columns after renaming:", df.columns)

    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'Cost ($)':row['Cost'],
                'fuel_costs_per_km': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            # Get demand for this combination
            demand = vehicles[0]['demand']  # All vehicles in the same group have the same demand
            
            # Calculate max vehicles based on the vehicle with highest yearly range
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            
            # Calculate max vehicles needed and round up to ensure demand is met
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        
        return max_vehicles

    def calculate_utilization(self, demand: float, num_vehicles: int, yearly_range: float) -> float:
        """Calculate utilization metric for a vehicle type"""
        if num_vehicles == 0 or yearly_range == 0:
            return 0
        return (demand / num_vehicles) / yearly_range * 100

    def calculate_demand_fulfillment(self, num_vehicles: int, max_vehicles: int) -> float:
        """Calculate demand fulfillment by fuel type"""
        if max_vehicles == 0:
            return 0
        return num_vehicles / max_vehicles

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the total number of vehicles in the solution is within the dynamic maximum limit"""
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    def calculate_total_cost(self, num_vehicles: int, vehicle: Dict) -> float:
        """Calculate total cost including insurance, maintenance, and fuel costs"""
        return num_vehicles * (
            vehicle['insurance_cost'] + 
            vehicle['maintenance_cost'] + 
            vehicle['fuel_costs_per_km']+
            vehicle['Cost ($)']
        )

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            # Generate a solution
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            
            # Randomly distribute vehicles while respecting the total maximum
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:  # 50% chance to add a vehicle
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions while respecting dynamic vehicle limit"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        # If we can't create valid children, return copies of parents
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.1) -> Dict:
        """Mutate a solution while respecting the dynamic vehicle limit"""
        attempts = 0
        max_attempts = 10
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            total_vehicles = sum(mutated_solution.values())
            
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if (change == 1 and total_vehicles < max_vehicles) or \
                       (change == -1 and mutated_solution[vehicle_type] > 0):
                        mutated_solution[vehicle_type] += change
                        total_vehicles += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution  # Return original if no valid mutation found

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness of a solution based on:
        1. Meeting demand requirements (significantly increased priority)
        2. Total cost
        3. Penalty for exceeding maximum vehicles
        """
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_cost = 0
        total_capacity = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_cost += self.calculate_total_cost(num_vehicles, vehicle)
            total_capacity += num_vehicles * vehicle['yearly_range']
        
        # Get demand for this combination
        demand = vehicles[0]['demand']
        
        # Calculate demand fulfillment ratio (1.0 = fully met, < 1.0 = not fully met)
        demand_fulfillment = min(1.0, total_capacity / demand) if demand > 0 else 1.0
        
        # Much stronger penalty for not meeting demand (increased by 10x)
        demand_penalty = max(0, demand - total_capacity) * 10000
        
        # Calculate demand bonus to favor solutions that meet demand
        demand_bonus = 5000 * demand_fulfillment
        
        # Normalize costs (lower is better)
        cost_score = 1000 / (total_cost + 1)
        
        # Return fitness with much higher weight on meeting demand
        return demand_bonus + cost_score - demand_penalty

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            # Calculate fitness for all solutions
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            # Sort by fitness (descending)
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            # Update best solution if needed
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            # Select parents for next generation
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            # Create next generation
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results in the required format"""
        results = []
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            # Calculate total capacity and demand for this group
            vehicles = self.vehicles_by_size_distance[size_distance]
            demand = vehicles[0]['demand']
            total_capacity = 0
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    total_capacity += num_vehicles * vehicle_data['yearly_range']
                    total_cost = self.calculate_total_cost(num_vehicles, vehicle_data)

                    # Calculate evaluation metrics
                    utilization = self.calculate_utilization(
                        vehicle_data['demand'],
                        num_vehicles,
                        vehicle_data['yearly_range']
                    )
                    
                    demand_fulfillment = self.calculate_demand_fulfillment(
                        num_vehicles,
                        max_vehicles
                    )

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Cost ($)": round(total_cost, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Utilization": round(utilization, 2),
                        "Demand_Fulfillment": round(demand_fulfillment, 2)
                    })
            
            # Print warning if demand is not met for this group
            demand_met_percentage = (total_capacity / demand) * 100 if demand > 0 else 100
            if demand_met_percentage < 100:
                print(f"WARNING: Demand not fully met for {size_distance}. Capacity: {total_capacity}, Demand: {demand} ({demand_met_percentage:.1f}%)")

        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with dynamic maximum vehicles...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    
    # Calculate and print the total cost across all allocations
    total_cost = optimized_results["Cost ($)"].sum()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    print(f"\nTotal Fleet Cost: ${total_cost:,.2f}")
    
    # Check if all demand is met
    allocations = optimized_results["Allocation"].unique()
    total_demand_met = True
    
    for allocation in allocations:
        allocation_rows = optimized_results[optimized_results["Allocation"] == allocation]
        demand = allocation_rows["Demand"].iloc[0]
        total_capacity = sum(allocation_rows["no_of_vehicles"] * allocation_rows["Yearly Range"])
        if total_capacity < demand:
            total_demand_met = False
            print(f"WARNING: {allocation} - Demand not fully met. Capacity: {total_capacity}, Demand: {demand}")
    
    if total_demand_met:
        print("\nAll demand requirements have been successfully met!")
    else:
        print("\nWARNING: Some demand requirements were not fully met. Consider adjusting parameters.")
    
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"  # Replace with your CSV file path
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_2023.csv", index=False)


Running optimization with dynamic maximum vehicles...
Columns after renaming: Index(['Allocation', 'Operating Year', 'Size', 'Distance_demand', 'Demand',
       'ID', 'Vehicle', 'Available Year', 'Cost', 'Yearly_Range',
       'Distance_vehicle', 'Fuel', 'carbon_emissions_per_km', 'Insurance_Cost',
       'Maintenance_Cost', 'Fuel_Costs', 'Total_Cost', 'Topsis_Score', 'Rank',
       'Distance'],
      dtype='object')

Optimized Fleet Allocation:
              Allocation Vehicle    Cost ($) Fuel  no_of_vehicles  \
0   Size S1, Distance D1  Diesel   450501.36  B20               5   
1   Size S1, Distance D2  Diesel   360401.09  B20               4   
2   Size S1, Distance D2     LNG   954001.51  LNG               9   
3   Size S1, Distance D3  Diesel   991102.99  B20              11   
4   Size S1, Distance D3     LNG   636001.00  LNG               6   
5   Size S1, Distance D4  Diesel   270300.82  B20               3   
6   Size S2, Distance D1  Diesel   551201.37  B20               5 